In [ ]:

# Necessary imports - done
# ------------------------
import os
from PIL import Image
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random
import copy
import cv2
from os import listdir
from os.path import isfile, join
import h5py
import time
import multiprocessing
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import math
import sys
from tqdm import tqdm
import pandas as pd
import csv
import json
import imageio

import torch
from torch.autograd import Variable
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as vutils
from torchvision.utils import save_image
import torch.optim as optim

from __future__ import print_function
from io import BytesIO


# scipy related
# -------------
import scipy
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import filters
from scipy import misc


# necessary imports
# -----------------
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial import distance_matrix


# working now
# -----------
#import skimage.io
#from skimage.transform import rotate, AffineTransform, warp
#from skimage.util import random_noise
#from skimage.filters import gaussian
#from skimage import transform as tf


# neccessary imports for imgaug
# ------------------------------
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

###
###
%matplotlib inline
%env JOBLIB_TEMP_FOLDER=/tmp


# printing platform info
# ----------------------
import platform
print(platform.python_version())

# code

## helper functions

In [ ]:
# GENERIC - change an torch image to numpy image
# ----------------------------------------------
def to_numpy_image(xin):
    
    try:
        xin = xin.data.numpy()
    except:
        xin = xin.numpy()
    
    xout = np.swapaxes(xin,1,2)
    xout = np.swapaxes(xout,2,3)
    
    # returns axes swapped numpy images
    # ---------------------------------
    return xout       



# GENERIC - converts numpy images to torch tensors for training
# -------------------------------------------------------------
def setup_image_tensor(xin):
    xout = np.swapaxes(xin,1,3)
    xout = np.swapaxes(xout,2,3)
    
    # returns axes swapped torch tensor
    # ---------------------------------
    xout = torch.from_numpy(xout)
    return xout.float()


In [ ]:
# a simple non pool function that takes in main dict and indices
# --------------------------------------------------------------


def save_cluster_imgs_pool(out_folder_in,index_list_in,name_index_in):
    
    # 0. inits
    # --------
    global name_index
    name_index = name_index_in
    
    global index_list
    index_list = index_list_in
    
    global out_folder
    out_folder = out_folder_in
    
    # 2. calling pool function
    # ------------------------
    pool = ThreadPool(10)
    pool.map(save_cluster_imgs_single, list(range(len(index_list_in))))
    print('Done with function save.')
    
    # closing pools
    # -------------
    pool.terminate()
    pool.join()
    
    
def save_cluster_imgs_single(i):
    
    # calling globals
    # ---------------
    global name_index
    global index_list
    global out_folder
    global x_dict_out
    
    # saving
    # ------
    img_name = out_folder + '/' + name_index + '_' + str(i) + '.jpg'
    Image.fromarray(x_dict_out[index_list[i]]).save(img_name)
    
    
    
    
    
    

In [ ]:
def save_cluster_to_folder(cluster_labels,no_images,save_folder,name):
    
    # a simple loop
    # -------------
    for c in range(np.max(cluster_labels)+1):
        
        # creating cluster list
        # ---------------------
        curr_clus_inds = list(np.argwhere(clustering.labels_==c)[:,0])
        curr_clus_inds = curr_clus_inds[0:no_images]
        
        # calling save function
        # ---------------------
        save_cluster_imgs_pool(save_folder,curr_clus_inds,name+'_cluster_'+str(c))
        
        print('done with current cluster: ' + str(c) + ' of ' + str(np.max(clustering.labels_)+1) + ' clusters..')
        
    
    

In [ ]:
# a function to load a saved model
# --------------------------------

def load_saved_model_function(path, use_cuda):
    
    
    ''' path = /folder1/folder2/model_ae.tar format'''
    
    # 1. loading full model
    # ---------------------
    model = torch.load(path.replace('.tar','_MODEL.tar'))
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,model.parameters()))
    
    # 2. Applying state dict
    # ----------------------
    if use_cuda == True:
        
        # loads to GPU
        # ------------
        checkpoint = torch.load(path)
        
    else:
        # loads to CPU
        # ------------
        checkpoint = torch.load(path, map_location=lambda storage, loc: storage)
        
        
    # loading checkpoint
    # -------------------
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # loading optimizer
    # -----------------
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if use_cuda == True:
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.cuda()
            
            
            
    # loading other stuff
    # -------------------
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    loss_mode = checkpoint['loss_mode']
    
    return model, optimizer, epoch, loss, loss_mode
    
    


In [ ]:
# simple function to read a single image
# --------------------------------------

def create_dataset_from_folder_all(infolder,resize,gray_mode,n_h,n_w):
    
    # 0. global initialisations
    # -------------------------
    global x_dict
    x_dict = {}
    
    global index_list
    index_list = []
    
    global resize_flag
    resize_flag = resize
    
    global gb_in_folder
    gb_in_folder = infolder

    global counter
    counter = 0
    
    global new_h
    new_h = n_h
    
    global new_w
    new_w = n_w
    
    global image_list
    image_list_jpg = [f for f in listdir(infolder) if isfile(join(infolder, f)) and '.jpg' in f.lower()]
    image_list_png = [f for f in listdir(infolder) if isfile(join(infolder, f)) and '.png' in f.lower()]
    image_list = image_list_jpg + image_list_png

    global x_images_dataset
    x_images_dataset = np.zeros((len(image_list),new_h,new_w,3), dtype='uint8')
    
    global x_images_dataset_gray
    x_images_dataset_gray = np.zeros((len(image_list),new_h,new_w), dtype='uint8')
    
    global x_images_dataset_edge
    x_images_dataset_edge = np.zeros((len(image_list),new_h,new_w,1))
    
    
    # 1.1 sanity assertion
    # -------------------
    assert len(image_list) > 0, 'No images in the folder'
    
    
    # 2. calling resize function across multiprocessing pool
    # ------------------------------------------------------
    pool = ThreadPool(5) 
    pool.map(create_dataset_from_folder_single, list(range(len(image_list))))
    
    # 2.1 sanity assert
    # -----------------
    assert x_images_dataset.shape[0] == x_images_dataset_gray.shape[0], 'RGB and Grayscale images have different numbers of images!'
    
    # 3. filtering out the dataset
    # ----------------------------
    print('Len at start: ' + str(x_images_dataset.shape))
    x_images_dataset = x_images_dataset[index_list]
    x_images_dataset_gray = x_images_dataset_gray[index_list]
    x_images_dataset_edge = x_images_dataset_edge[index_list]
    
    # 4. index correcting dict
    # ------------------------
    global x_dict_out
    x_dict_out = {}
    for i in range(len(index_list)):
        x_dict_out[i] = x_dict[index_list[i]]
        
    
    
    # hard normalising grayscale dataset by individual means
    # ------------------------------------------------------
    if gray_mode == 'bw':
        
        # hard b/w single channel
        # -----------------------        
        mn = np.mean(np.mean(x_images_dataset_gray, axis = 1), axis = 1)
        mn = mn.reshape(mn.shape[0],1,1)
        x_images_dataset_gray[x_images_dataset_gray < mn] = 0
        x_images_dataset_gray[x_images_dataset_gray >= mn] = 255
        x_images_dataset_gray = x_images_dataset_gray.reshape(x_images_dataset_gray.shape[0],new_h,new_w,1)
        
    elif gray_mode == 'gray_3':
        
        # grayscale 3 channel
        # -------------------
        x_images_dataset_gray = x_images_dataset_gray.reshape(x_images_dataset_gray.shape[0],new_h,new_w,1)
        x_images_dataset_gray = np.concatenate((x_images_dataset_gray,x_images_dataset_gray,x_images_dataset_gray), axis= 3)
        
    else:
        
        # grayscale 1 channel
        # -------------------
        x_images_dataset_gray = x_images_dataset_gray.reshape(x_images_dataset_gray.shape[0],new_h,new_w,1)
        

    print('Len after filtering: ' + str(x_images_dataset.shape))
    print('Done creating dataset of around ' + str(counter) + ' images. Access them at global x_images_dataset, x_images_dataset_gray, x_images_dataset_edge, x_dict_out.')
    
    # closing pools
    # -------------
    pool.terminate()
    pool.join()
    

def create_dataset_from_folder_single(i):
    
    # 0. calling global variables
    # ---------------------------
    global gb_in_folder
    global counter
    global new_h
    global new_w
    global x_images_dataset
    global x_images_dataset_gray
    global image_list
    global resize_flag
    global x_images_dataset_edge
    global x_dict
    
    # 1. ops
    # ------
    try:
        name = image_list[i]
        img_main = cv2.imread(join(gb_in_folder, name))
        img_main_rbg = cv2.cvtColor(copy.deepcopy(img_main), cv2.COLOR_BGR2RGB)
        img = cv2.cvtColor(copy.deepcopy(img_main), cv2.COLOR_BGR2RGB)
        img_gray = cv2.cvtColor(copy.deepcopy(img_main), cv2.COLOR_BGR2GRAY)
        
        # resizing ops
        # -----------
        if resize_flag == True:
            img = cv2.resize(img, (new_w,new_h))
            img_gray = cv2.resize(img_gray, (new_w,new_h))
            

        # 5. by default building edge images
        # ----------------------------------
        blurred = cv2.GaussianBlur(img_gray.reshape(new_h,new_w).astype('uint8'), (7, 7), 0)
        edged = cv2.Canny(blurred, 50, 150)
        edged = edged.reshape(new_h,new_w,1)
        
        # final assignments
        # ------------------
        x_images_dataset[i] = img
        x_images_dataset_gray[i] = img_gray
        x_images_dataset_edge[i] = edged
        
        counter += 1
        index_list.append(i)
        x_dict[i] = img_main_rbg
    
    except:
        
        # do nothing
        ##
        pass
    


In [ ]:
# a simple forward pass function
# ------------------------------


def simple_forward_pass_pool(xin,input_is_image,model):
    
    
    # 0. initialisations
    # ------------------
    global model_global
    model_global = model
    
    global x_in_global
    x_in_global = copy.deepcopy(xin)
    
    # 1. setting up y_out size
    # ------------------------
    global y_out_global
    sz = list(model(xin[0:2]).size())
    final_size = sz[1:]
    final_size = tuple([xin.size()[0]] + final_size)
    y_out_global = torch.zeros((final_size))
    
    
    # 1.1 sanity
    # ----------
    if input_is_image == True:
        assert torch.mean(x_in_global) > 1, 'Error: Data already in 0-1 range'
        x_in_global = x_in_global/torch.max(x_in_global)
        
    
    # 2. calling pool function
    # ------------------------
    pool = ThreadPool(10)
    pool.map(simple_forward_pass_single, list(range(xin.size()[0])))
    print('Done with forward pass of around ' + str(xin.size()[0]) + ' examples. Access them at global y_out_global.')
    
    # closing pools
    # -------------
    pool.terminate()
    pool.join()
    
    
    
def simple_forward_pass_single(i):
    
    # 0. global inits
    # ---------------
    global model_global
    global x_in_global
    global y_out_global
    
    # 1. ops
    # ------
    curr_example = x_in_global[i]
    curr_example = curr_example.view(1,curr_example.size()[0],curr_example.size()[1],curr_example.size()[2])
    curr_out = model_global(curr_example)
    y_out_global[i] = curr_out[0]
    
    

In [ ]:
# function that will return final latents for similarity function 
# ---------------------------------------------------------------

def final_latents(f_maps,kernel_stride_dims,pool_mode,aggregate_pool_maps):
    
    
    '''
    
    1. input is a dict with keys - 
    
    a. f_maps - list of feature maps (m,c,h,w) on which pooling functions can be run for latent computation
    b. kernel_dims - list of kernel dimensions that will be used for pooling on feature maps
    c. pool_mode - 'max', 'avg' or 'both'
    d. aggregate_pool_maps - either sum up pool values or not
    
    2. output will be a dict with final latents to be input to similarity function
    3. output latents will be L2 normalized
    
    
    '''

    
    # 0. initialisations
    # ------------------
    latents_out = []
    
    
    # 1. computing rmac latents
    # -------------------------
    for each_fmap in f_maps:
        for each_kernel_dim in kernel_stride_dims:
            curr_latent = return_ms_rmac([each_fmap],pool_mode,each_kernel_dim,aggregate_pool_maps).data.numpy()
            
            # L2 normalization
            # ----------------
            #curr_latent = curr_latent / np.linalg.norm(curr_latent)
            
            # appending
            # ----------
            latents_out.append(curr_latent)
            print('done at kernel ' + str(each_kernel_dim) + '. latent size: ' + str(curr_latent.shape))
    
    
    
    # final return
    # ------------
    return latents_out



In [ ]:
# main function
# -------------

def return_ms_rmac(fmaps_list,pool_mode,kernel_dims,aggregate_pool_maps):
    
    '''
    ref: https://www.researchgate.net/publication/313465134_MS-RMAC_Multiscale_Regional_Maximum_Activation_of_Convolutions_for_Image_Retrieval
    
    1. takes as input fmaps tuple with feature masps of size (m,k,h,w) each where k = no_channels at each layer
    -- input is ((m1,k1,h1,w1), (m2,k2,h2,w2),...)
    -- pool_mode = 'max','avg','both'
    -- kernel_dims = None or (f,s)
    -- aggregate_pool_maps - if true, we will sum across channels, else leave as it is
    
    2. works out 3 scales of MAC kernel sizes for each feature map in tuple
    3. computes MAC i.e., maximum activations convolutions & aggregates them
    4. returns concatenated (m,K) vector where K = k1 + k2 + k3.. i.e., no_channels at each layer in fmaps in tuple
    
    '''
    
    # 0. initialisations
    # -------------------
    assert type(fmaps_list) == list, 'Type error: input feature maps must be a list'
    
    
    # 1. looping through fmap tuple
    # -----------------------------
    for fmap in fmaps_list:
        
        # 1.1 checking kernel initialisations
        # -----------------------------------
        if kernel_dims == None:
            
            # 1.1.1 we will be using preset scaled regions - computing f,s
            # ------------------------------------------------------------
            h,w = fmap.size()[2],fmap.size()[3]

            # scale 1 : w is same, h = h/2, stride = curr_h/3
            ##
            l1_h = int(h/2)
            l1_w = w
            l1_s = int(l1_h/2)

            # scale 2 : h = h/2, w = 2w/3, stride = curr_w/2
            ##
            l2_h = int(h/2)
            l2_w = int(2*w/3)
            l2_s = int(l2_w/2)

            # scale 3 : h = 2h/5, w = w/2, stride = curr_w/2
            ##
            l3_h = int(2*h/5)
            l3_w = int(w/2)
            l3_s = int(l3_w/2)
            
            
            # 1.1.1.2 forward pass to get pool maps
            # --------------------------------------
            if pool_mode == 'max':
                
                # computing mx pool @ scale 1
                # ---------------------------
                pl_l1 =  nn.Sequential(*[nn.MaxPool2d((l1_h,l1_w), stride=l1_s)])
                l1_pool_map = pl_l1(fmap)
                l1_pool_map = torch.sum(torch.sum(l1_pool_map, 2),2)
                
                # computing mx pool @ scale 2
                # ---------------------------
                pl_l2 =  nn.Sequential(*[nn.MaxPool2d((l2_h,l2_w), stride=l2_s)])
                l2_pool_map = pl_l2(fmap)
                l2_pool_map = torch.sum(torch.sum(l2_pool_map, 2),2)
                
                # computing mx pool @ scale 3
                # ---------------------------
                pl_l3 =  nn.Sequential(*[nn.MaxPool2d((l3_h,l3_w), stride=l3_s)])
                l3_pool_map = pl_l3(fmap)
                l3_pool_map = torch.sum(torch.sum(l3_pool_map, 2),2)
                
                
                # NOT concatenating -- summing
                # -----------------------------
                combined_pool_map = l1_pool_map + l2_pool_map + l3_pool_map
                
                
            
            elif pool_mode == 'avg':
                

                # computing avg pool @ scale 1
                # ---------------------------
                pl_l1 =  nn.Sequential(*[nn.AvgPool2d((l1_h,l1_w), stride=l1_s)])
                l1_pool_map = pl_l1(fmap)
                l1_pool_map = torch.sum(torch.sum(l1_pool_map, 2),2)
                
                # computing avg pool @ scale 2
                # ---------------------------
                pl_l2 =  nn.Sequential(*[nn.AvgPool2d((l2_h,l2_w), stride=l2_s)])
                l2_pool_map = pl_l2(fmap)
                l2_pool_map = torch.sum(torch.sum(l2_pool_map, 2),2)
                
                # computing avg pool @ scale 3
                # ---------------------------
                pl_l3 =  nn.Sequential(*[nn.AvgPool2d((l3_h,l3_w), stride=l3_s)])
                l3_pool_map = pl_l3(fmap)
                l3_pool_map = torch.sum(torch.sum(l3_pool_map, 2),2)
                
                # NOT concatenating -- summing
                # ----------------------------
                combined_pool_map = l1_pool_map + l2_pool_map + l3_pool_map
                
                
            elif pool_mode == 'both':
                
                # computing mx pool @ scale 1
                # ---------------------------
                pl_l1 =  nn.Sequential(*[nn.MaxPool2d((l1_h,l1_w), stride=l1_s)])
                l1_pool_map = pl_l1(fmap)
                l1_pool_map = torch.sum(torch.sum(l1_pool_map, 2),2)
                
                # computing mx pool @ scale 2
                # ---------------------------
                pl_l2 =  nn.Sequential(*[nn.MaxPool2d((l2_h,l2_w), stride=l2_s)])
                l2_pool_map = pl_l2(fmap)
                l2_pool_map = torch.sum(torch.sum(l2_pool_map, 2),2)
                
                # computing mx pool @ scale 3
                # ---------------------------
                pl_l3 =  nn.Sequential(*[nn.MaxPool2d((l3_h,l3_w), stride=l3_s)])
                l3_pool_map = pl_l3(fmap)
                l3_pool_map = torch.sum(torch.sum(l3_pool_map, 2),2)
                
                
                # computing avg pool @ scale 1
                # ---------------------------
                avg_pl_l1 =  nn.Sequential(*[nn.AvgPool2d((l1_h,l1_w), stride=l1_s)])
                avg_l1_pool_map = avg_pl_l1(fmap)
                avg_l1_pool_map = torch.sum(torch.sum(avg_l1_pool_map, 2),2)
                
                # computing avg pool @ scale 2
                # ---------------------------
                avg_pl_l2 =  nn.Sequential(*[nn.AvgPool2d((l2_h,l2_w), stride=l2_s)])
                avg_l2_pool_map = avg_pl_l2(fmap)
                avg_l2_pool_map = torch.sum(torch.sum(avg_l2_pool_map, 2),2)
                
                # computing avg pool @ scale 3
                # ---------------------------
                avg_pl_l3 =  nn.Sequential(*[nn.AvgPool2d((l3_h,l3_w), stride=l3_s)])
                avg_l3_pool_map = avg_pl_l3(fmap)
                avg_l3_pool_map = torch.sum(torch.sum(avg_l3_pool_map, 2),2)
                
                
                # NOT concatenating -- summing
                # ----------------------------
                combined_pool_map = l1_pool_map + l2_pool_map + l3_pool_map + avg_l1_pool_map + avg_l2_pool_map + avg_l3_pool_map

            else:
                
                assert 1 == 2,'Error: invalid pool mode'
                
            
            # 1.1.1.3 finally concatenating pool maps across all input feature maps
            # ---------------------------------------------------------------------
            try:
                all_layers_pool_map = torch.cat((all_layers_pool_map,combined_pool_map), 1)
            except:
                all_layers_pool_map = combined_pool_map
            
            
            
            
        # if the input dims are given
        # ---------------------------
        else:
            
            
            # 1.1.2 dims is given as (f,s)
            # ----------------------------
            curr_f = kernel_dims[0]
            curr_s = kernel_dims[1]
            
            # 1.1.2.1 setting up sequentials
            # ------------------------------
            mxpl =  nn.Sequential(*[nn.MaxPool2d((curr_f,curr_f), stride=curr_s)])
            avgpl =  nn.Sequential(*[nn.AvgPool2d((curr_f,curr_f), stride=curr_s)])
            
            
            # 1.1.2.2 forward pass to get pool maps
            # --------------------------------------
            if pool_mode == 'max':
                
                # computing max pool
                # ------------------
                pool_map = mxpl(fmap)

            
            elif pool_mode == 'avg':
                
                # computing avg pool
                # ------------------
                pool_map = avgpl(fmap)
            
            elif pool_mode == 'both':
                
                # computing both
                # --------------
                mx_pool_map = mxpl(fmap)
                avg_pool_map = avgpl(fmap)
                
                # concatenating
                # -------------
                #pool_map = torch.cat((mx_pool_map,avg_pool_map), 1)
                pool_map = mx_pool_map + avg_pool_map
                
            else:
                
                assert 1 == 2,'Error: invalid pool mode'
            
            
            
            # 1.1.2.3 aggregating
            # -------------------
            if aggregate_pool_maps == True:
                
                # summing along h,w axises - pool_map will be of shape (m,no_channels)
                # --------------------------------------------------------------------
                pool_map = torch.sum(torch.sum(pool_map, 2),2)
                
            
            # final resize before concat
            # --------------------------
            pool_map = pool_map.view(pool_map.size()[0],-1)
            
            
            # final concat along channel axis
            # --------------------------------
            try:
                all_layers_pool_map = torch.cat((all_layers_pool_map,pool_map), 1)
            except:
                all_layers_pool_map = pool_map
                

    # final return
    # ------------
    return all_layers_pool_map
            


# models

In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_ae_4_layer_std(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2) # nn.Tanh() nn.Softmax2d()
        f = 3
        s = 2
        dropout_prob = 0.1
        dropout_node = nn.Dropout2d(p=dropout_prob)
        
        #  what worked - 3,64,128,256,128,64,3
        
        # Conv 1
        ###
        conv1 = 64
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1)
        
        # Conv 2
        ###
        conv2 = 128
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2)
        
        # Conv 3
        ###
        conv3 = 256
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv #nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3)
        
        # Conv 4
        ###
        conv4 = 512
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nn.Softmax2d()#nw_activation_conv #nn.Softmax2d() #nw_activation_conv
        cl4 = [ct4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) # size 6 x 4
        
        
        # Pooling layer
        #mxpl =  [nn.MaxPool2d((2,2), stride=2)]
        #avpl =  [nn.AvgPool2d((6,4), stride=1)]
        #self.pool_net = nn.Sequential(*mxpl)
        
        # Adding a fully connected linear layer
        #
        
        # Upconv layer 0
        ###
        #t0 = nn.ConvTranspose2d(conv4,conv4,2,stride = 2)
        #b0 = nn.BatchNorm2d(conv4)
        #a0 = nw_activation_conv
        #l0 = [t0,b0,a0]
        #self.upcl0 = nn.Sequential(*l0)
        
        # Upconv layer 1
        ###
        up_conv1 = 256
        t1 = nn.ConvTranspose2d(conv4,up_conv1,f,stride = s)
        b1 = nn.BatchNorm2d(up_conv1)
        a1 = nw_activation_conv
        l1 = [t1,b1,a1,dropout_node]
        self.upcl1 = nn.Sequential(*l1)
        
        # Upconv layer 2
        ###
        up_conv2 = 128
        t2 = nn.ConvTranspose2d(up_conv1,up_conv2,f,stride = s)
        b2 = nn.BatchNorm2d(up_conv2)
        a2 = nw_activation_conv
        l2 = [t2,b2,a2,dropout_node]
        self.upcl2 = nn.Sequential(*l2)
        
        # Upconv layer 3
        ###
        up_conv3 = 64
        t3 = nn.ConvTranspose2d(up_conv2,up_conv3,f,stride = s)
        b3 = nn.BatchNorm2d(up_conv3)
        a3 = nw_activation_conv
        l3 = [t3,b3,a3,dropout_node]
        self.upcl3 = nn.Sequential(*l3)
        
        # Upconv layer 4
        ###
        t4 = nn.ConvTranspose2d(up_conv3,in_channels,f,stride = s)
        a4 = nn.Sigmoid()
        l4 = [t4,a4]
        self.upcl4 = nn.Sequential(*l4)
        

    def forward(self, x):
        
        # Generation
        # ----------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        c4_out = self.convl4(c3_out)
        #c5_out = self.pool_net(c3_out)
        
        #f1_out = self.upcl0(c5_out)
        f2_out = self.upcl1(c4_out)
        f3_out = self.upcl2(f2_out)
        f4_out = self.upcl3(f3_out)
        f5_out = self.upcl4(f4_out)
        
        return f5_out

    
    def latent(self, x):
        
        
        # Generation
        # ----------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        c4_out = self.convl4(c3_out)
            
        
        return c4_out
        



In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_ae_3_layer_std(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2) # nn.Tanh() nn.Softmax2d()
        f = 3
        s = 2
        dropout_prob = 0.1
        dropout_node = nn.Dropout2d(p=dropout_prob)
        
        #  what worked - 3,64,128,256,128,64,3
        
        # Conv 1
        ###
        conv1 = 32
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1)
        
        # Conv 2
        ###
        conv2 = 64
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2)
        
        # Conv 3
        ###
        conv3 = 128
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nn.Softmax2d() #nw_activation_conv
        cl3 = [ct3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3)
        
        # Conv 4
        ###
        #conv4 = 256
        #ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        #cb4 = nn.BatchNorm2d(conv4)
        #ca4 = nn.Softmax2d() #nw_activation_conv
        #cl4 = [ct4,ca4,dropout_node]
        #self.convl4 = nn.Sequential(*cl4) # size 6 x 4
        
        
        # Pooling layer
        #mxpl =  [nn.MaxPool2d((2,2), stride=2)]
        #avpl =  [nn.AvgPool2d((6,4), stride=1)]
        #self.pool_net = nn.Sequential(*mxpl)
        
        # Adding a fully connected linear layer
        #
        
        # Upconv layer 0
        ###
        #t0 = nn.ConvTranspose2d(conv4,conv4,2,stride = 2)
        #b0 = nn.BatchNorm2d(conv4)
        #a0 = nw_activation_conv
        #l0 = [t0,b0,a0]
        #self.upcl0 = nn.Sequential(*l0)
        
        # Upconv layer 1
        ###
        #up_conv1 = 128
        #t1 = nn.ConvTranspose2d(conv4,up_conv1,f,stride = s)
        #b1 = nn.BatchNorm2d(up_conv1)
        #a1 = nw_activation_conv
        #l1 = [t1,b1,a1,dropout_node]
        #self.upcl1 = nn.Sequential(*l1)
        
        # Upconv layer 2
        ###
        up_conv2 = 64
        t2 = nn.ConvTranspose2d(conv3,up_conv2,f,stride = s)
        b2 = nn.BatchNorm2d(up_conv2)
        a2 = nw_activation_conv
        l2 = [t2,b2,a2,dropout_node]
        self.upcl2 = nn.Sequential(*l2)
        
        # Upconv layer 3
        ###
        up_conv3 = 32
        t3 = nn.ConvTranspose2d(up_conv2,up_conv3,f,stride = s)
        b3 = nn.BatchNorm2d(up_conv3)
        a3 = nw_activation_conv
        l3 = [t3,b3,a3,dropout_node]
        self.upcl3 = nn.Sequential(*l3)
        
        # Upconv layer 4
        ###
        t4 = nn.ConvTranspose2d(up_conv3,in_channels,f,stride = s)
        a4 = nn.Sigmoid()
        l4 = [t4,a4]
        self.upcl4 = nn.Sequential(*l4)
        

    def forward(self, x):
        
        # Generation
        # ----------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        #c4_out = self.convl4(c3_out)
        #c5_out = self.pool_net(c3_out)
        
        #f1_out = self.upcl0(c5_out)
        #f2_out = self.upcl1(c4_out)
        f3_out = self.upcl2(c3_out)
        f4_out = self.upcl3(f3_out)
        f5_out = self.upcl4(f4_out)
        
        return f5_out

    
    def latent(self, x):
        
        
        # 0. forward prop
        # ---------------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        latent_out = self.convl3(c2_out)
            
        
        return latent_out
        



In [ ]:
# END OF CODE
##

# executions

In [ ]:
# URL set ups for execution
# -------------------------

## SET UP CUDA OR NOT HERE + OTHER SET UPS
##########################################

dev_env = 'local' # 'gpu' or 'local'

##########################################
##########################################

# Setting CUDA
# ------------
if dev_env == 'gpu':
    use_cuda = True
else:
    use_cuda = False
if use_cuda == True:
    torch.cuda.empty_cache()
    

# SET FILE SPECIFIC NAMES HERE
# ----------------------------
if dev_env == 'gpu':
    save_path = '/home/venkateshmadhava/codes/pmate2_localgpuenv/models/'
    parent_url = '/home/venkateshmadhava/datasets/images'
else:
    save_path = '/Users/venkateshmadhava/Documents/pmate2/pmate2_env/models/'
    parent_url = '/Users/venkateshmadhava/Documents/pmate2/local_working_files'


# displaying save path
# --------------------
print(save_path)

### 1. loading data

In [ ]:
# step 1. creating dataset from folder
# ------------------------------------
in_folder = '/Users/venkateshmadhava/Documents/si_projects/datasets/si_dataset_1'


# 1.1 creating dataset
# --------------------
create_dataset_from_folder_all(in_folder,True,'gray_3',175,175)
global x_images_dataset, x_images_dataset_gray, x_images_dataset_edge


# 1.2 main setups -- DO NOT CHANGE
# --------------------------------
use_grayscale = True
use_edge = False
if use_grayscale == True:
    if use_edge == True:
        xtest = x_images_dataset_edge
    else:
        xtest = x_images_dataset_gray
else:
    xtest = x_images_dataset

    
# 1.3 for printing view
# ---------------------
xtest_print = x_images_dataset


# 1.4 sanity
# ----------
del x_images_dataset, x_images_dataset_gray, x_images_dataset_edge
print(len(x_dict_out))
print(xtest.shape)

In [ ]:
# sanity viewing DO THIS ALWAYS
# ------------------------------

randrange = random.sample(list(range(xtest.shape[0])), 2)

for i in randrange:
    print(i)
    if use_grayscale == True:
        plt.imshow(xtest[i,:,:,0],cmap='gray')
        plt.show()
    else:
        plt.imshow(xtest[i])
        plt.show()

### 2. loading model and checking reconstruction

In [ ]:
# loading saved model
# -------------------

#torch.nn.Module.dump_patches = False
cn_file_name = 'ae_model_tboard_3_layer_128_latent_gray_softmax.tar'#'ae_model_tboard_4_layer_512_softmax_RGB.tar'
cn_save_path = save_path + cn_file_name
model,_,_,_,_ = load_saved_model_function(cn_save_path, use_cuda)
model = model.eval()
model

In [ ]:
# setting up data
# ---------------

xtrn = Variable(setup_image_tensor(xtest[:,:,:,0:1])).float()
print(xtrn.size())
print(torch.mean(xtrn))

In [ ]:
# checking reconstruction -- ALWAYS DO THIS
# -----------------------------------------
randrange = random.sample(list(range(xtrn.size()[0])), 2)
recons = model.eval()(xtrn[randrange]/torch.max(xtrn[randrange]))
recons = to_numpy_image(recons)


# showing images
# --------------
for i in range(2):
    
    print('orig: ')
    plt.imshow(xtest[randrange[i]])
    plt.show()
    print('recons: ')
    plt.imshow(recons[i,:,:,0],cmap='gray')
    plt.show()
    

### 3. extracting feature maps & preparing latents

In [ ]:
# extracting feature maps
# -----------------------

simple_forward_pass_pool(xtrn,True,model.latent)
global y_out_global
input_fmaps = y_out_global
del y_out_global
print(input_fmaps.size())

In [ ]:
################# LATENT SETTINGS #################
###################################################

# what works RGB -- RGB local - (1/3*h,1/3*w) i.e if h,w of fmap is 10,10 then local pool dim is 3,3

# latents settings
# ----------------
kernel_stride_dims = [[5,2]]
pool_mode = 'both'
aggregate_pool_maps = False

In [ ]:
# computing latents from fmaps
# ----------------------------
input_latents_list = final_latents([input_fmaps],kernel_stride_dims,pool_mode,aggregate_pool_maps)

# final step before clustering
# ----------------------------
input_latents = input_latents_list[0]
input_latents.shape

### 4. kmeans

In [ ]:
# use this for checking hyper params -- DO NOT DELETE
# ---------------------------------------------------

#X = np.array([[1, 2], [1, 4], [1, 0],[4, 2], [4, 4], [4, 0]])
#print(X.shape)

In [ ]:
# main code for clustering -- use 1.5 as default
# this will take time... approximately 2 minutes for a dataset of 1000 images
# ---------------------------------------------------------------------------

dist_threshold = np.mean(distance_matrix(input_latents,input_latents)) * 1.5
clustering = AgglomerativeClustering(n_clusters=None,distance_threshold=dist_threshold,compute_full_tree=True).fit(input_latents)
print('total clusters found: ' + str(np.max(clustering.labels_)+1))

In [ ]:
# visualising clusters
# --------------------

for c in range(np.max(clustering.labels_)+1):
    
    print('current cluster: ' + str(c))
    curr_clus_inds = list(np.argwhere(clustering.labels_==c)[:,0])
    print('no of images here: ' + str(len(curr_clus_inds)))
    
    
    # just showing limted images
    # --------------------------
    fig=plt.figure(figsize=(30, 30)) # in format (r,c)
    columns = 5
    rows = 5
    counter = 0

    for each_ind in curr_clus_inds:
        
        counter += 1
        fig.add_subplot(rows, columns, counter)
        plt.imshow(xtest[each_ind])
        
        if counter == 25:
            break
    plt.show()
    
    print('*********************************')



### 5. saving to folder

In [ ]:
# final saving to an output folder
# set save params here
# ---------------------------------

file_name_prefix = 'random_noise_si_dtset_2'
max_no_images_per_cluster_to_save = 2
out_folder = '/Users/venkateshmadhava/Desktop/tempp'

In [ ]:
# run this for final save -- this will save 
# max_no_images_per_cluster_to_save number of images per cluster to the
# out folder
# ----------------------------------------------------------------------


save_cluster_to_folder(clustering.labels_,max_no_images_per_cluster_to_save,out_folder,file_name_prefix)

In [ ]:
# bird, dog, fish, star, leaves, sun, mountains, flowers

# rough